# 2D to 3D Lifting Network

## Imports and Setup

In [1]:
import os
import src.bug_dataset
from src.models.simple_3d_model import LinearModel
from src.train_lifting_network import Train_LiftNetwork
import numpy as np
import pandas as pd
from skimage import io
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
plt.ion()   # interactive mode

In [2]:
# Plotting Code
%matplotlib widget
import matplotlib.pyplot as plt

def plot_stick_bug(ax, points, vis, prediction=False):
    limb_ranges=[[0,7],[8,14],[15,21],[22,28],[29,35],[36,42],[43,49],[53,56],[59,62]]
    reduced_limb_ranges = [[0,4],[4,7],[7,10],[10,13],[13,16],[16,19],[19,22],[22,25],[25,28]]
    if len(points) == 62:
        points = np.array(points).T
        for num in range(len(limb_ranges)):
            visible_limb = []
            for x in range(limb_ranges[num][0],limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, label='Prediction', color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], label='Actual', color='blue')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4, color='blue')
        return ax, line
    elif len(points) == 28:
        points = np.array(points).T
        for num in range(len(reduced_limb_ranges)):
            visible_limb = []
            for x in range(reduced_limb_ranges[num][0],reduced_limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, label='Prediction', color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], label='Actual', color='blue')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4, color='blue')
        return ax, line

def set_axes_equal(ax):
    # workaround, as matplotlib's 3D plot has no option for equisised axes (10/2021)
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])


In [3]:
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))

In [4]:
reduceKeypoints = True
sungaya_dataset = src.bug_dataset.BugDataset(df=out_df,
                             root_dir=target_dir, reduced=reduceKeypoints, transform=transforms.Compose([
                                src.bug_dataset.ToTensor()
                                   ]))

In [5]:
train_split = 0.7
valid_split = 0.1
train_size = int(train_split * len(sungaya_dataset))
valid_size = int(valid_split * len(sungaya_dataset))
test_size = len(sungaya_dataset) - (train_size+valid_size)
train_dataset,valid_dataset ,test_dataset = torch.utils.data.random_split(sungaya_dataset, [train_size, valid_size, test_size], generator=torch.Generator().manual_seed(42))

In [6]:
batch_size = 128

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [7]:
if reduceKeypoints:
    model = LinearModel(2*28,28*3,1024) # Linear size 512 looked promising 256 too small
else:
    model = LinearModel(2*62,62,512)

epochs = 200
learning_rate = 1e-4

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
liftingtrainer = Train_LiftNetwork(model, optimizer, train_dataloader, valid_dataloader, sungaya_dataset.std_3d, sungaya_dataset.means_3d)
trained_model = liftingtrainer.run(epochs)
# https://github.com/weigq/3d_pose_baseline_pytorch Re look over this 

Using cuda device


Lifting Model Epochs:   0%|          | 0/2000 [00:00<?, ?it/s]

Finished Epoch 001: | Train Acc: 0.049 | Train Loss: 0.62109 | Val Acc: 0.000 | Val Loss: 0.00000
Finished Epoch 002: | Train Acc: 0.046 | Train Loss: 0.43813 | Val Acc: 0.000 | Val Loss: 0.00000
Finished Epoch 003: | Train Acc: 0.046 | Train Loss: 0.43648 | Val Acc: 0.000 | Val Loss: 0.00000
Finished Epoch 004: | Train Acc: 0.054 | Train Loss: 0.38846 | Val Acc: 0.000 | Val Loss: 0.00000
Finished Epoch 005: | Train Acc: 0.057 | Train Loss: 0.33641 | Val Acc: 0.000 | Val Loss: 0.00000
Finished Epoch 006: | Train Acc: 0.051 | Train Loss: 0.35663 | Val Acc: 0.000 | Val Loss: 0.00000
Finished Epoch 007: | Train Acc: 0.051 | Train Loss: 0.33143 | Val Acc: 0.000 | Val Loss: 0.00000
Finished Epoch 008: | Train Acc: 0.049 | Train Loss: 0.29189 | Val Acc: 0.000 | Val Loss: 0.00000
Finished Epoch 009: | Train Acc: 0.059 | Train Loss: 0.30340 | Val Acc: 0.000 | Val Loss: 0.00000
Finished Epoch 010: | Train Acc: 0.068 | Train Loss: 0.30505 | Val Acc: 0.000 | Val Loss: 0.00000
Finished Epoch 011: 

In [8]:
trained_model = liftingtrainer.model
trained_model.eval()
# Code get the first batch of results
with torch.no_grad():
    for data in train_dataloader:
        image = data['file_name']
        X = data['key_points_2D']
        y = data['key_points_3D']
        vis = data['visibility']
        # print(y)
        X, y = X.to(liftingtrainer.device, dtype=torch.float), y.to(liftingtrainer.device, dtype=torch.float)
        
        pred = trained_model(torch.flatten(X, start_dim=1, end_dim=2))
        break

In [19]:
sample = 7
prediction = (torch.reshape(pred,(len(pred),28,3))[sample].cpu()).numpy()
actual = (y[sample].cpu()).numpy()
visibility_mask = vis[sample]

mean = sungaya_dataset.means_3d
std = sungaya_dataset.std_3d

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

unnormalised_acc = (actual*sungaya_dataset.std_3d)+sungaya_dataset.means_3d
ax, line1 = plot_stick_bug(ax, unnormalised_acc, visibility_mask)

unnormalised_est = (prediction*sungaya_dataset.std_3d)+sungaya_dataset.means_3d
# unnormalised_est = unnormalised_acc.copy()
# unnormalised_est[:,2] = (prediction*std)+mean
ax, line2 = plot_stick_bug(ax, unnormalised_est, visibility_mask, True)

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
plt.title(image[sample])
plt.legend(handles=[line1,line2], loc='upper right')

# use custom function to ensure equal axis proportions
set_axes_equal(ax)
# opens external plot
plt.show()
print(unnormalised_acc[2])
print(unnormalised_est[2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[0. 0. 0.]
[0. 0. 0.]


In [10]:
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
# PATH = "lift.pt"
# torch.save(trained_model.state_dict(), PATH)